<a href="https://colab.research.google.com/github/bereml/riiaa-20-mtl/blob/master/notebooks/1a_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconocimiento de acciones humanas (4 pts.)

En este ejercicio debes comparar arquitecturas RNN y CNN para reconocimiento de acciones humanas en el conjunto UCF11. La solución debe cumplir con los siguientes puntos.

* Usar las características convolucionales vistas en clase.
* Implementar una arquitectura RNN bidireccional con una capa GRU.
* Implementar una arquitectura CNN con una capa Conv1d.
* Modificar el tamaño de las capas para que ambos modelos tengan un número similar de parámetros.
* Discutir el comportamiento durante el entrenamiento y resultados finales en ambos conjuntos.


# Reconocimiento de acciones humanas usando RNNs 

Curso: [Aprendizaje Profundo](http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_profundo/). Profesor: [Gibran Fuentes Pineda](http://turing.iimas.unam.mx/~gibranfp/). Ayudantes: [Bere](https://turing.iimas.unam.mx/~bereml/) y [Ricardo](https://turing.iimas.unam.mx/~ricardoml/).


---
---

En esta libreta entrenaremos un modelo basado en RNNs para reconocimiento de acciones humanas (HAR) en el conjunto [UCF11](https://www.crcv.ucf.edu/data/UCF_YouTube_Action.php).

<img src="https://www.crcv.ucf.edu/data/youtube_snaps.jpg" width=800/>

Este ejemplo está basado en las ideas presentadas en [*Long-term Recurrent Convolutional Networks for Visual Recognition and Description*](https://arxiv.org/abs/1411.4389) de 2016 por Donahue et al. 

## 1 Preparación

### 1.1 Bibliotecas

In [1]:
# Colab
# https://github.com/TylerYep/torchinfo
!pip install torchinfo
# https://zarr.readthedocs.io/en/stable/
!pip install zarr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 kB 854.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 14.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5050 sha256=d185d0e0058141474c2bfb8e2355558a51273726df19491b67291d9999e2d8ef
  Stored in directory: /root/.cache/pip/wheels/12/1c/38/0def51e15add93bff3f4bf9c248b94db0839b980b8535e72a0
Successfully built asciitree


In [2]:
# sistema de archivos
import os
# funciones aleatorias
import random
# descomprimir
import tarfile
# sistema de archivos
from os.path import join

# arreglos multidimensionales
import numpy as np
# redes neuronales
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets.utils as tvu
# almacenamiento de arreglos multidimensionales
import zarr
#redes
from torch.utils.data import DataLoader, random_split
# inspección de arquitectura
from torchinfo import summary

# directorio de datos
DATA_DIR = '../data'

# tamaño del lote
BATCH_SIZE = 32
# tamaño del vector de características
FEAT_SIZE = 1024

# reproducibilidad
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch_gen = torch.manual_seed(SEED)

## 2 Datos

### 2.1 Conjunto de datos

In [3]:
class UCF11:

    def __init__(self, root, download=False):
        self.root = root
        self.zarr_dir = join(root, 'ucf11.zarr')
        if download:
            self.download()
        self.z = zarr.open(self.zarr_dir, 'r')
        self.paths = list(self.z.array_keys())
        
    def __getitem__(self, i):
        arr = self.z[self.paths[i]]
        # [10, 1024] 10 cuadros por 1024 características cada uno
        x = np.array(arr)
        # [1] etiqueta de salida ej. número del 1 al 10 posibles etiquetas 
        y = np.array(arr.attrs['y'], dtype=np.int64)
        return x, y

    def __len__(self):
        return len(self.paths)
    
    def _check_integrity(self):
        return os.path.isdir(self.zarr_dir)
    
    def _extract(self, root, filename):
        tar = tarfile.open(join(root, filename), "r:gz")
        tar.extractall(root)
        tar.close()

    def download(self):
        if self._check_integrity():
            print('Files already downloaded and verified')
            return
        tvu.download_url(
            url='https://cloud.xibalba.com.mx/s/apYrNA4iM4K65o7/download',
            root=self.root,
            filename='ucf11.zarr.tar.gz',
            md5='c8a82454f9ec092d00bcd99c849e03fd'
        )
        self._extract(self.root, 'ucf11.zarr.tar.gz')

### 2.2 Instancia del conjunto y partición

In [4]:
ds = UCF11(join(DATA_DIR, 'ucf11'), True)
x, y = ds[0]
print(f'x shape={x.shape} dtype={x.dtype}')
print(f'x [0][:5]={x[0][:5]}')
print(f'y shape={y.shape} dtype={y.dtype} {y}')
print(f'y {y}')

  0%|          | 0/53436566 [00:00<?, ?it/s]

x shape=(10, 1024) dtype=float32
x [0][:5]=[0.00022111 0.00368518 0.00314753 0.00201778 0.09296297]
y shape=() dtype=int64 0
y 0


In [5]:
trn_size = int(0.8 * len(ds))
tst_size = len(ds) - trn_size
trn_ds, tst_ds = random_split(ds, [trn_size, tst_size])
len(trn_ds), len(tst_ds)

(1279, 320)

### 2.3 Cargadores de datos

In [6]:
trn_dl = DataLoader(
    # conjunto
    trn_ds,
    # tamaño del lote
    batch_size=BATCH_SIZE,
    # desordenar
    shuffle=True,
    # procesos paralelos
    num_workers=2
)
tst_dl = DataLoader(
    # conjunto
    tst_ds,
    # tamaño del lote
    batch_size=BATCH_SIZE,
    # desordenar
    shuffle=True,
    # procesos paralelos
    num_workers=2
)

In [7]:
x, y = next(iter(trn_dl))
print(f'x shape={x.shape} dtype={x.dtype}')
print(f'y shape={y.shape} dtype={y.dtype}')

x shape=torch.Size([32, 10, 1024]) dtype=torch.float32
y shape=torch.Size([32]) dtype=torch.int64


## 3 Modelo

<!-- Torchvision provee una familia de [modelos](https://pytorch.org/docs/1.6.0/torchvision/models.html#classification) preentrenados en ImageNet. Usaremos [Shufflenet V2](https://arxiv.org/abs/1807.11164), una arquitectura eficiente para clasificación de imágenes.  -->

### 3.1 Definición de arquitectura

### 3.1.1 Definición de arquitectura bidireccional RNN

<center><img src="https://miro.medium.com/max/764/1*6QnPUSv_t9BY9Fv8_aLb-Q.png" width="500"/></center>

In [8]:
class RNN(nn.Module):

    def __init__(self, input_size=1024, hidden_size=128, num_classes=11):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_size)
        # input_size es el tamaño de la secuencia de entrada
        # hidden_size es el tamaño de la memoria interna de la celda y que produce a la salida
        # num_layers es el número de capaz que se apilan
        # batch_first es que espera el tamaño del lote al inicio
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=1, batch_first=True, bidirectional=True)
        self.cls = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        # Batch, Seq, Feats, Hidden
        # [32, 10, 1024]
        
        # [B, S, F] => [B, F, S]
        x = x.movedim(1, 2)
        # [B, F, S]
        x = self.bn(x)
        # [B, F, S] => [B, S, F]
        x = x.movedim(1, 2)
        # [B, S, F] => [B, S, H]
        # [32, 10, 1024] => [32, 10, 128*2 (hidden_size*2)]
        x, _ = self.rnn(x)
        # [B, S, H] => [B, H]
        # [32, 10, 256] => [32, 256] toma el último paso.
        # Se pueden aplicar distintas ténicas de submuestreo
        # 1. Tomar el último paso de la salida
        #x = x[:, -1, :] #Tomar todo el lote, el último paso de la secuencia, y todas las características.
        x = torch.mean(x,1) #2. Promedio
        #x = torch.max(x,1) #3. Max pooling
        # [B, H] = [B, 11]
        x = self.cls(x)
        return x

In [9]:
modelRNN = RNN().eval()
modelRNN(torch.zeros(1, 10, 1024)).shape

torch.Size([1, 11])

In [10]:
summary(modelRNN, (1, 10, 1024), device='cpu', verbose=0)

Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      [1, 11]                   --
├─BatchNorm1d: 1-1                       [1, 1024, 10]             2,048
├─GRU: 1-2                               [1, 10, 256]              886,272
├─Linear: 1-3                            [1, 11]                   2,827
Total params: 891,147
Trainable params: 891,147
Non-trainable params: 0
Total mult-adds (M): 8.87
Input size (MB): 0.04
Forward/backward pass size (MB): 0.10
Params size (MB): 3.56
Estimated Total Size (MB): 3.71

## 3.1.2 Definición de la arquitectura CNN

In [11]:
class CNN(nn.Module):

    def __init__(self, input_size=1024, in_channels=10, num_classes=11):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.cnn = nn.Sequential(
            # bloque conv1
            # [B, S, F]
            # [32, 10, 1024] => [32, 10, 1024]
            nn.Conv1d(in_channels=10, out_channels=10, kernel_size=3, padding=1),
            # [32, 10, 1024] = [32, 10, 1024]
            nn.ReLU(),
            # [32, 10, 1024] => [32, 10, 256]
            nn.MaxPool2d(kernel_size=2, stride=4),
        )
        self.cls = nn.Linear(256, num_classes)

    def forward(self, x):
        # Batch, Seq, Feats, Hidden
        # [32, 10, 1024]
        
        # [B, S, F] => [B, F, S]
        x = x.movedim(1, 2)
        # [B, F, S]
        x = self.bn(x)
        # [B, F, S] => [B, S, F]
        x = x.movedim(1, 2)
        # [B, S, F] => [B, S, H]
        # [32, 10, 1024] => [32, 10, 256]
        x = self.cnn(x)
        # [B, S, H] => [B, H]
        # [32, 10, 256] => [32, 256] toma el último paso.
        # Se pueden aplicar distintas ténicas de submuestreo
        # 1. Tomar el último paso de la salida
        # 2. 
        #x = x[:, -1, :] #Tomar todo el lote, el último paso de la secuencia, y todas las características.
        x = torch.mean(x,1) #Promedio
        #x = torch.max(x,1) #Max pooling
        # [B, H] = [B, 11]
        x = self.cls(x)
        return x

In [12]:
modelCNN = CNN().eval()
modelCNN(torch.zeros(1, 10, 1024)).shape

torch.Size([1, 11])

In [13]:
summary(modelCNN, (1, 10, 1024), device='cpu', verbose=0)

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [1, 11]                   --
├─BatchNorm1d: 1-1                       [1, 1024, 10]             2,048
├─Sequential: 1-2                        [1, 3, 256]               --
│    └─Conv1d: 2-1                       [1, 10, 1024]             310
│    └─ReLU: 2-2                         [1, 10, 1024]             --
│    └─MaxPool2d: 2-3                    [1, 3, 256]               --
├─Linear: 1-3                            [1, 11]                   2,827
Total params: 5,185
Trainable params: 5,185
Non-trainable params: 0
Total mult-adds (M): 0.32
Input size (MB): 0.04
Forward/backward pass size (MB): 0.16
Params size (MB): 0.02
Estimated Total Size (MB): 0.23

## 4 Entrenamiento

### 4.1 Ciclo de entrenamiento

### 4.1.1 Entrenamiento con RNN

In [14]:
# optimizador
opt = optim.Adam(modelRNN.parameters(), lr=1e-3)

# ciclo de entrenamiento
EPOCHS = 10
for epoch in range(EPOCHS):

    # modelo en modo de entrenamiento
    modelRNN.train()
    
    # entrenamiento de una época
    for x, y_true in trn_dl:
        # hacemos inferencia para obtener los logits
        y_lgts = modelRNN(x)
        # calculamos la pérdida
        loss = F.cross_entropy(y_lgts, y_true)
        # vaciamos los gradientes
        opt.zero_grad()
        # retropropagamos
        loss.backward()
        # actulizamos parámetros
        opt.step()

    # desactivamos temporalmente la gráfica de cómputo
    with torch.no_grad():

        # modelo en modo de evaluación
        modelRNN.eval()
        
        losses, accs = [], []
        # validación de la época
        for x, y_true in tst_dl:
            # hacemos inferencia para obtener los logits
            y_lgts = modelRNN(x)
            # calculamos las probabilidades
            y_prob = F.softmax(y_lgts, 1)
            # obtenemos la clase predicha
            y_pred = torch.argmax(y_prob, 1)
            
            # calculamos la pérdida
            loss = F.cross_entropy(y_lgts, y_true)
            # calculamos la exactitud
            acc = (y_true == y_pred).type(torch.float32).mean()

            # guardamos históricos
            losses.append(loss.item() * 100)
            accs.append(acc.item() * 100)

        # imprimimos métricas
        loss = np.mean(losses)
        acc = np.mean(accs)
        print(f'E{epoch:2} loss={loss:6.2f} acc={acc:.2f}')

E 0 loss=190.82 acc=37.19
E 1 loss=161.14 acc=48.44
E 2 loss=147.69 acc=52.81
E 3 loss=140.66 acc=54.38
E 4 loss=132.86 acc=57.81
E 5 loss=126.65 acc=60.31
E 6 loss=129.13 acc=61.25
E 7 loss=132.95 acc=60.31
E 8 loss=127.48 acc=61.25
E 9 loss=134.98 acc=60.31


### 4.1.2 Entrenamiento con CNN

In [15]:
# optimizador
opt = optim.Adam(modelCNN.parameters(), lr=1e-3)

# ciclo de entrenamiento
EPOCHS = 10
for epoch in range(EPOCHS):

    # modelo en modo de entrenamiento
    modelCNN.train()
    
    # entrenamiento de una época
    for x, y_true in trn_dl:
        # hacemos inferencia para obtener los logits
        y_lgts = modelCNN(x)
        # calculamos la pérdida
        loss = F.cross_entropy(y_lgts, y_true)
        # vaciamos los gradientes
        opt.zero_grad()
        # retropropagamos
        loss.backward()
        # actulizamos parámetros
        opt.step()

    # desactivamos temporalmente la gráfica de cómputo
    with torch.no_grad():

        # modelo en modo de evaluación
        modelCNN.eval()
        
        losses, accs = [], []
        # validación de la época
        for x, y_true in tst_dl:
            # hacemos inferencia para obtener los logits
            y_lgts = modelCNN(x)
            # calculamos las probabilidades
            y_prob = F.softmax(y_lgts, 1)
            # obtenemos la clase predicha
            y_pred = torch.argmax(y_prob, 1)
            
            # calculamos la pérdida
            loss = F.cross_entropy(y_lgts, y_true)
            # calculamos la exactitud
            acc = (y_true == y_pred).type(torch.float32).mean()

            # guardamos históricos
            losses.append(loss.item() * 100)
            accs.append(acc.item() * 100)

        # imprimimos métricas
        loss = np.mean(losses)
        acc = np.mean(accs)
        print(f'E{epoch:2} loss={loss:6.2f} acc={acc:.2f}')

E 0 loss=234.68 acc=16.56
E 1 loss=224.12 acc=24.06
E 2 loss=210.69 acc=30.00
E 3 loss=198.80 acc=35.31
E 4 loss=192.71 acc=36.25
E 5 loss=185.33 acc=39.38
E 6 loss=177.46 acc=38.75
E 7 loss=170.46 acc=44.69
E 8 loss=168.46 acc=43.44
E 9 loss=162.34 acc=43.75


## 5. Modificación para que CNN tenga un número silimar de parámetros que la RNN (891,147)

### 5.1 Definición de la nueva arquitectura CNN

In [16]:
class CNN2(nn.Module):

    def __init__(self, input_size=1024, in_channels=10, num_classes=11):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.cnn = nn.Sequential(
            # bloque conv1
            # [B, S, F]
            # [32, 10, 1024] => [32, 10, 1024]
            nn.Conv1d(in_channels=10, out_channels=10, kernel_size=3, padding=1),
            # [32, 10, 1024] = [32, 10, 1024]
            nn.ReLU(),
            # [32, 10, 1024] => [32, 10, 1024]
            nn.MaxPool2d(kernel_size=3, stride=1),
        )
        self.cls0 = nn.Linear(1022, 512)
        self.relu1 = nn.ReLU()
        self.cls1 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.cls = nn.Linear(256, num_classes)

    def forward(self, x):
        # Batch, Seq, Feats, Hidden
        # [32, 10, 1024]
        
        # [B, S, F] => [B, F, S]
        x = x.movedim(1, 2)
        # [B, F, S]
        x = self.bn(x)
        # [B, F, S] => [B, S, F]
        x = x.movedim(1, 2)
        # [B, S, F] => [B, S, H]
        # [32, 10, 1024] => [32, 10, 1024]
        x = self.cnn(x)
        # [B, S, H] => [B, H]
        # [32, 10, 256] => [32, 256] toma el último paso.
        # Se pueden aplicar distintas ténicas de submuestreo
        # 1. Tomar el último paso de la salida
        # 2. 
        #x = x[:, -1, :] #Tomar todo el lote, el último paso de la secuencia, y todas las características.
        x = torch.mean(x,1) #Promedio
        #x = torch.max(x,1) #Max pooling
        # [B, H] = [B, 11]
        x = self.cls0(x)
        x = self.relu1(x)
        x = self.cls1(x)
        x = self.relu2(x)
        x = self.cls(x)
        return x

In [17]:
modelCNN2 = CNN2().eval()
modelCNN2(torch.zeros(1, 10, 1024)).shape

torch.Size([1, 11])

In [18]:
summary(modelCNN2, (1, 10, 1024), device='cpu', verbose=0)

Layer (type:depth-idx)                   Output Shape              Param #
CNN2                                     [1, 11]                   --
├─BatchNorm1d: 1-1                       [1, 1024, 10]             2,048
├─Sequential: 1-2                        [1, 8, 1022]              --
│    └─Conv1d: 2-1                       [1, 10, 1024]             310
│    └─ReLU: 2-2                         [1, 10, 1024]             --
│    └─MaxPool2d: 2-3                    [1, 8, 1022]              --
├─Linear: 1-3                            [1, 512]                  523,776
├─ReLU: 1-4                              [1, 512]                  --
├─Linear: 1-5                            [1, 256]                  131,328
├─ReLU: 1-6                              [1, 256]                  --
├─Linear: 1-7                            [1, 11]                   2,827
Total params: 660,289
Trainable params: 660,289
Non-trainable params: 0
Total mult-adds (M): 0.98
Input size (MB): 0.04
Forward/backward p

### 5.2 Entrenamiento con CNN2 (660,289)

In [19]:
# optimizador
opt = optim.Adam(modelCNN2.parameters(), lr=1e-3)

# ciclo de entrenamiento
EPOCHS = 10
for epoch in range(EPOCHS):

    # modelo en modo de entrenamiento
    modelCNN2.train()
    
    # entrenamiento de una época
    for x, y_true in trn_dl:
        # hacemos inferencia para obtener los logits
        y_lgts = modelCNN2(x)
        # calculamos la pérdida
        loss = F.cross_entropy(y_lgts, y_true)
        # vaciamos los gradientes
        opt.zero_grad()
        # retropropagamos
        loss.backward()
        # actulizamos parámetros
        opt.step()

    # desactivamos temporalmente la gráfica de cómputo
    with torch.no_grad():

        # modelo en modo de evaluación
        modelCNN2.eval()
        
        losses, accs = [], []
        # validación de la época
        for x, y_true in tst_dl:
            # hacemos inferencia para obtener los logits
            y_lgts = modelCNN2(x)
            # calculamos las probabilidades
            y_prob = F.softmax(y_lgts, 1)
            # obtenemos la clase predicha
            y_pred = torch.argmax(y_prob, 1)
            
            # calculamos la pérdida
            loss = F.cross_entropy(y_lgts, y_true)
            # calculamos la exactitud
            acc = (y_true == y_pred).type(torch.float32).mean()

            # guardamos históricos
            losses.append(loss.item() * 100)
            accs.append(acc.item() * 100)

        # imprimimos métricas
        loss = np.mean(losses)
        acc = np.mean(accs)
        print(f'E{epoch:2} loss={loss:6.2f} acc={acc:.2f}')

E 0 loss=231.82 acc=22.50
E 1 loss=221.68 acc=21.88
E 2 loss=193.49 acc=36.56
E 3 loss=181.37 acc=38.44
E 4 loss=173.38 acc=43.75
E 5 loss=158.50 acc=43.75
E 6 loss=153.21 acc=51.56
E 7 loss=148.79 acc=50.62
E 8 loss=155.33 acc=52.19
E 9 loss=142.54 acc=54.38


# Conclusiones y discusión

La red neuronal recurrente (RNN) bidireccional con una capa GRU tiene en total 891,147 parámetros entrenables. La red neuronal convolucional (CNN) tiene un total de 5,185 parámetros entrenables.
  
La RNN para el conjunto de prueba llega a una exactitud de 60.31. Y la CNN obtiene una exactitud de 43.75. Ambas en la época número 10.
  
Una de las razones por la que la RNN obtuvo una mejor métrica se debe a que se aprovecha la temporalidad de la información recibida, ya que es una secuencia de las características extraídas de 10 cuadros del video. Además, a pesar de que la secuencia es larga (1024) y que puede ocurrir desvanecimiento del gradiente, la RNN sigue obtiendo un buen rendimiento comparada con la CNN. También, se podría pensar que por el número de parámetros de la RNN puede llegar a existir sobre-ajuste, sin embargo, desde la época 5 parece que ya no se modifica la exactitud.
  
Por otro lado, para lograr incrementar el número de parámetros de la red convolucional, se agregaron capas ocultas a la capa densa con el objetivo. Llegando a un total de 660,289 parámetros entrenables. Al entrenar este segundo modelo se obtiene una exactitud de 54.38 en el conjunto de prueba, y compararlo con la RNN, se concluye que el número de parámetros mejoró la exactitud de la CNN, sin embargo, la RNN sigue obteniendo un mejor rendimiento.